In [3]:
import pandas as pd
import numpy as np
from prophet import Prophet

# Carga de datos
df = pd.read_csv('sell-in.txt', sep='\t')
df['fecha'] = pd.to_datetime(df['periodo'].astype(str), format='%Y%m')

# Función para crear la serie de tiempo para un producto
def serie_producto(id_producto):
    df_prod = df[df['product_id'] == id_producto].loc[:, ['fecha', 'tn']]
    if len(df_prod) == 0:
        return None
    
    # Filtrar solo los períodos de 2019
    df_prod = df_prod[df_prod['fecha'].dt.year == 2019]

    # Encontrar la fecha inicial y final para el producto
    fecha_inicial = df_prod['fecha'].min()
    fecha_final = df_prod['fecha'].max()
    
    # Generar fechas hasta febrero de 2020
    fechas_productos = pd.date_range(start=fecha_inicial, end='2020-02-01', freq='MS')
    df_fechas_productos = pd.DataFrame({'fecha': fechas_productos})
    
    df_prod = df_prod.groupby('fecha').agg({'tn': 'sum'}).reset_index()
    df_ret = pd.merge(df_fechas_productos, df_prod, on='fecha', how='left')
    df_ret['tn'].interpolate(method='linear', inplace=True)
    df_ret.fillna(0, inplace=True)
    return df_ret

# DataFrame para almacenar predicciones
df_pred = pd.read_csv('productos_a_predecir.txt', sep='\t')

# Predicciones
resultados = []

for i in range(len(df_pred)):
    id_producto = df_pred.iloc[i, 0]
    df_prod = serie_producto(id_producto)
    print(f"Progress: {i+1}/{len(df_pred)}", end="\r")
    if df_prod is not None:
        df_prod_prophet = df_prod.rename(columns={'fecha': 'ds', 'tn': 'y'})
        
        modelo = Prophet()
        modelo.fit(df_prod_prophet)
        
        future = modelo.make_future_dataframe(periods=1, freq='MS')
        forecast = modelo.predict(future)
        
        pred_final = abs(forecast['yhat'].iloc[-1])  # Valor absoluto para evitar negativos
        resultados.append([id_producto, pred_final])

# Crear DataFrame con los resultados
df_resultados = pd.DataFrame(resultados, columns=['product_id', 'tn'])

# Guardar predicciones
df_resultados.to_csv('pred_prophet.csv', index=False)


ModuleNotFoundError: No module named 'Prophet'